# Chapter 2 - Gaussian mixture model


## Data generation

1. 10 means $m_k$ were drawn from a bivariate Gaussian distribution $\mathcal{N}((1,0),I)$ and labeled *BLUE*. 
2. 10 more were drawn from $\mathcal{N}((0,1),I)$ and labeled *ORANGE*. 
3. For each class 100 observations were generated as follows: 
   * $m_k$ was picked at random with probability 1/10;
   * observation was drawn from $\mathcal{N}(m_k,I/5)$, thus leading to a mixture of Gaussian clusters for each class.

In [ ]:
import numpy
from matplotlib import pyplot as plt

%matplotlib inline

# define commonly used colors
GRAY1 = '#231F20'
GRAY4 = '#646369'
PURPLE = '#A020F0'
BLUE =  '#57B5E8'
ORANGE = '#E69E00'

In [ ]:
# Generate the means
N_means = 10
N_data = 100
means_blue = numpy.random.multivariate_normal((1,0),[[1,0],[0,1]], N_means)
means_orange = numpy.random.multivariate_normal((0,1),[[1,0],[0,1]], N_means)
means_all = numpy.vstack((means_blue, means_orange))

# Generate the data
# Need to make this better rather than 
bdata = []
odata = []
for i in range(N_data):
    mb = means_blue[numpy.random.choice(10)]
    mo = means_orange[numpy.random.choice(10)]
    b1 = numpy.random.multivariate_normal(mb,[[1/5,0],[0,1/5]], 1)
    o1 = numpy.random.multivariate_normal(mo,[[1/5,0],[0,1/5]], 1)
    bdata.append(b1[0])
    odata.append(o1[0])
    
bdata = numpy.array(bdata)
odata = numpy.array(odata)

X_train = numpy.vstack((bdata, odata))
y_train = numpy.concatenate((numpy.zeros(N_data, dtype=numpy.int), numpy.zeros(N_data, dtype=numpy.int)+1))

# Plot the data
# prepares a plot with a title and circles representing training data
def plot_train_data(title):
    fig, ax = plt.subplots(figsize=(2.8, 2.8), dpi=110)
    ax.set_aspect(1.3)
    ax.scatter(X_train[:, 0], X_train[:, 1], s=18, facecolors='none', edgecolors=numpy.array([BLUE, ORANGE])[y_train].flatten())
    ax.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
    ax.set_xlim(-2.6, 4.2)
    ax.set_ylim(-2.0, 2.9)
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    ax.text(-2.6, 3.2, title, color=GRAY4, fontsize=9)
    for spine in ax.spines.values():
        spine.set_color(GRAY1)
    return fig, ax


# test it
_, _ = plot_train_data('Training data')


In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.mixture.gaussian_mixture import _compute_precision_cholesky
import pandas

# even though we already know means and covariances, we need to
# do "fake fit", otherwise GaussianMixture model will not work
gaussian_mixture_model = GaussianMixture(
    n_components=20,
    covariance_type='spherical',
    means_init=means_all,
    random_state=1
).fit(means_all)
# set known covariances
gaussian_mixture_model.covariances_ = [1/5]*20

In [ ]:
# This is hacky! 
# GaussianMixture uses precisions_cholesky_ for predict_proba method. 
# We need to recalculate precisions_cholesky_ since we changed covariances_.
gaussian_mixture_model.precisions_cholesky_ = _compute_precision_cholesky(
    gaussian_mixture_model.covariances_,  
    gaussian_mixture_model.covariance_type
)

In [ ]:
# Sample 10000 points for testing
X_test, y_test = gaussian_mixture_model.sample(10000)

# y_test contains sampled component indices
# index < 10 means that the class is BLUE (0)
y_test = 1*(y_test >= 10)

## Setup some plotting functions

In [ ]:
# given a model prediction function computes X points on n x n grid and the
# corresponding predicted classes
def fill_prediction_grid(n1, n2, predict):
    x1, x2 = numpy.linspace(-2.6, 4.2, n1), numpy.linspace(-2.0, 2.9, n2)
    X = numpy.transpose([numpy.tile(x1, n2), numpy.repeat(x2, n1)])
    y = predict(X)
    return X, y


# given a model prediction function computes X0 and X1 n x n meshgrids
# and the corresponing predicted classes meshgrid
def fill_prediction_meshgrid(predict):
    n1, n2 = 1000, 1000
    X, y = fill_prediction_grid(n1, n2, predict)
    return X[:, 0].reshape(n1, n2), X[:, 1].reshape(n1, n2), y.reshape(n1, n2)


# given a model prediction function plots train data, model decision
# bounary and background dots
def plot_model(predict, title):
    fig, ax = plot_train_data(title)
    # plot background dots
    X, y = fill_prediction_grid(69, 99, predict)
    ax.scatter(X[:, 0], X[:, 1], marker='.', lw=0, s=2,
               c=numpy.array([BLUE, ORANGE])[y])
    # plot the decision boundary
    X0, X1, Y = fill_prediction_meshgrid(predict)
    ax.contour(X0, X1, Y, [0.5], colors=GRAY1, linewidths=[0.7])
    return fig, ax

# given a model prediction function plots performance statistics
def plot_model_stat(predict, title, bayes=None):
    fig, ax = plot_model(predict, title)
    test_error_rate = 1 - accuracy_score(y_test, predict(X_test))
    train_error_rate = 1 - accuracy_score(y_train, predict(X_train))
    parms = {'color': GRAY1, 'fontsize': 7,
             'bbox': {'facecolor': 'white', 'pad': 3, 'edgecolor': 'none'}}
    ax.text(-2.42, -1.35, f'Training Error: {train_error_rate:.3f}', **parms)
    ax.text(-2.42, -1.62, f'Test Error:       {test_error_rate:.3f}', **parms)
    return fig, ax

<h2>Linear Regression</h2>

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# PAGE 12. Let’s look at an example of the linear model in a classiﬁcation
#          context.
linear_regression_model = LinearRegression().fit(X_train, y_train)


# PAGE 12. The fitted values Y-hat are converted to a fitted class variable
#          G-hat according to the rule G-hat = (ORANGE if Y-hat > 0.5, BLUE if
#          Y-hat ≤ 0.5.
def linear_predict(X):
    return 1*(linear_regression_model.predict(X) > 0.5)

# PAGE 13. The line is the decision boundary deﬁned by x.T @ b = 0.5. The
#          orange shaded region denotes that part of input space classiﬁed as
#          ORANGE, while the blue region is classiﬁed as BLUE.
_, _ = plot_model_stat(linear_predict, 'Linear Regression of 0/1 Response')

<h2>Nearest-Neighbor Methods</h2>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Run GridSearchCV to find the best n_neighbors parameter using the 10-folds
# CV. It finds 12, but the book uses 15-Nearest Neighbor Classifier because
# the authors selected the most parsimonious model within one standard error
# from the best model (one standard error rule). We will apply this rule in
# other examples, not here.
k_neighbors_grid_search = GridSearchCV(KNeighborsClassifier(),
                                       {'n_neighbors': list(range(1, 50))}, cv=10
                                      ).fit(X_train, y_train)
k_neighbors_grid_search.best_params_

In [ ]:
# PAGE 14. Use 15-nearest-neighbor averaging of the binary coded response as
#          the method of fitting. Thus Y-hat is the proportion of ORANGE’s in
#          the neighborhood, and so assigning class ORANGE to G-hat if
#          Y-hat>0.5 amounts to a majority vote in the neighborhood.
neighbors5_classifier = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)
_, _ = plot_model_stat(neighbors5_classifier.predict, '5-Nearest Neighbor Classifier')

In [ ]:
# PAGE 16. The classes are coded as a binary variable (BLUE = 0,ORANGE = 1),
#          and then predicted by 1-nearest-neighbor classiﬁcation.
neighbors1_classifier = KNeighborsClassifier(n_neighbors=1).fit(X_train, y_train)
_, _ = plot_model_stat(neighbors1_classifier.predict, '1−Nearest Neighbor Classifier')

## Optimal Bayes

In [ ]:
def optimal_bayes_predict(X):
    components_proba = gaussian_mixture_model.predict_proba(X)
    # first 10 components are BLUE(0), and others are BROWN(1)
    blue_proba = numpy.sum(components_proba[:, :10], axis=1)
    brown_proba = numpy.sum(components_proba[:, 10:], axis=1)
    y_hat = 1*(blue_proba < brown_proba)
    return y_hat

In [ ]:
bayes_error_rate = 1 - accuracy_score(y_test, optimal_bayes_predict(X_test))
print(f'The optimal Bayes error rate = {bayes_error_rate}')

In [ ]:
# plot the optimal Bayes decision boundary
_, _ = plot_model_stat(optimal_bayes_predict, 'Bayes Optimal Classifier')

In [ ]:
# lets save Bayes meshgrids for optimal decision boundary plotting
X0_bayes, X1_bayes, Y_bayes = fill_prediction_meshgrid(optimal_bayes_predict)